In [1]:
import tensorflow as tf
import keras_tuner as kt
import numpy as np
from tensorflow import keras

In [2]:
def model(hp):
    threshold = 0.5

    kernelPower1 = hp.Int("kernelPower1", min_value=1, max_value=8, step=1)
    kernelPower2 = hp.Int("kernelPower2", min_value=1, max_value=8, step=1)
    kernelPower3 = hp.Int("kernelPower3", min_value=1, max_value=8, step=1)
    kernelPower4 = hp.Int("kernelPower4", min_value=1, max_value=8, step=1)
    kernelPower5 = hp.Int("kernelPower5", min_value=1, max_value=8, step=1)
    
    
    METRICS = [
      #keras.metrics.TruePositives(name='tp', thresholds=threshold),
      #keras.metrics.FalsePositives(name='fp', thresholds=threshold),
      #keras.metrics.TrueNegatives(name='tn',thresholds=threshold),
      #keras.metrics.FalseNegatives(name='fn',thresholds=threshold), 
      keras.metrics.BinaryAccuracy(name='accuracy',threshold=threshold),
      keras.metrics.Precision(name='precision',thresholds=threshold),
      keras.metrics.Recall(name='recall',thresholds=threshold),
      #tfa.metrics.FBetaScore(name="fbeta", num_classes=1,beta=float(2),average='macro')
      #keras.metrics.SpecificityAtSensitivity(sensitivity=.95, name='SensAt95',threshold=threshold)
      #keras.metrics.AUC(name='auc'),
      #keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    ]
    
    model = tf.keras.models.Sequential([
        # This is the first convolution
        tf.keras.layers.Conv2D(2**kernelPower1, (3,3), activation='relu', input_shape=(128, 128, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(2**kernelPower2, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(2**kernelPower3, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The fourth convolution
        tf.keras.layers.Conv2D(2**kernelPower4, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # # The fifth convolution
        tf.keras.layers.Conv2D(2**kernelPower5, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(2**(kernelPower5), activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')])
                         
                         
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss="binary_crossentropy",
        metrics=METRICS,
    )
                         
    return(model)                     

In [3]:
image_size = (128, 128)
batch_size = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-training-and-validation",
    validation_split=0.25,
    subset="training",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode = "grayscale"
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-training-and-validation",
    validation_split=0.25,
    subset="validation",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode="grayscale"
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "tfData-testing",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    shuffle = True,
    color_mode="grayscale"
)

Found 17600 files belonging to 2 classes.
Using 13200 files for training.
Found 17600 files belonging to 2 classes.
Using 4400 files for validation.
Found 4401 files belonging to 2 classes.


In [4]:
tuner = kt.Hyperband(
    model,
    objective='val_loss',
    max_epochs=35)

In [5]:
tuner.search(train_ds, epochs=5, validation_data=val_ds)

Trial 90 Complete [00h 12m 04s]
val_loss: 0.2968235909938812

Best val_loss So Far: 0.04208162799477577
Total elapsed time: 04h 11m 12s
INFO:tensorflow:Oracle triggered exit


In [6]:
best_hyperparameters = tuner.get_best_hyperparameters()

In [8]:
for hyperparameters in best_hyperparameters:
    print(hyperparameters.get("kernelPower1"))
    print(hyperparameters.get("kernelPower2"))
    print(hyperparameters.get("kernelPower3"))
    print(hyperparameters.get("kernelPower4"))
    print(hyperparameters.get("kernelPower5"))

7
6
6
5
5
